<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Single Star Lightcurve with the Butler</b> <br>
Last verified to run on 2021-07-22 with LSST Science Pipelines release w_2021_25 <br>
Contact authors: Melissa Graham, Jeff Carlin <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a>. <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. <br>

**Credit:** This tutorial notebook was created by Jeff Carlin, Leanne Guy, and Melissa Graham, and draws on material in the fourth notebook "04 Intro to Butler".

### Caveats

The DP0.1 data set is not well suited to large-scale time domain analyses, and one goal of this notebook is to clearly illustrate the limitations.

The DP0.1 data set does not contain any data products from difference image analysis.

A catalog of sources detected in processed visit images (PVIs) is available via the Butler (read more in the [DP0.1 DPDD](https://dp0-1.lsst.io/data-products-dp0-1/index.html#dp0-1-data-products-definition-document-dpdd)).
However, this source catalog does not include *associations* of PVI sources by sky coordinate, nor associations between PVI sources and objects detected in the deep coadded images.

It is recommended that DP0 delegates work through the Butler notebooks "01 Intro to DP0 Notebooks" and "04 Intro to Butler" before using this notebook.

### Learning Objectives

First, use the TAP Service to find a time-domain target to make a light curve for.

Then, use the Butler to retrieve all detected sources from all processed visit images (PVIs or `calexps`) at a given sky coordinate, and plot magnitude as a function of time.

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_on

### Set Up

Import the packages we will need for this notebook.

In [ ]:
### Rubin-specific packages

import lsst.daf.butler as dafButler
import lsst.geom as geom
import lsst.sphgeom as sphgeom
import lsst.daf.base

# from rubin_jupyter_utils.lab.notebook import get_tap_service #, retrieve_query    
# service = get_tap_service()

### General python / astronomy packages

import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time

import time

Create an instance of the Butler.

In [ ]:
repo = 's3://butler-us-central1-dp01'
butler = dafButler.Butler(repo)
registry = butler.registry
collection = "2.2i/runs/DP0.1"

### 1. Use the TAP Service to choose a variable star.

Retrieve the RA and Dec of elements in the `truth_match` table that are within a 1 degree radius circle near the center of the DC2 region, and have `is_variable = 1` (is variable), `truth_type = 2` (is a star), and `mag_r > 20` (r-band AB magnitude is brighter than 20).
Find more information about the `truth_match` table in the [DP0.1 DPDD](https://dp0-1.lsst.io/data-products-dp0-1/index.html#dp0-1-data-products-definition-document-dpdd).

In [ ]:
# results = service.search("SELECT ra, dec "\
#                          "FROM dp01_dc2_catalogs.truth_match "\
#                          "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 62.0, -37.0, 1.0)) = 1 "\
#                          "AND is_variable = 1 AND truth_type = 2 AND mag_r > 20 ", maxrec=10000)

In [ ]:
## Alternative: use TAP service to choose a Type Ia supernova: truth_type = 3
# results = service.search("SELECT ra, dec, host_galaxy "\
#                          "FROM dp01_dc2_catalogs.truth_match "\
#                          "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 62.0, -37.0, 1.0)) = 1 "\
#                          "AND truth_type = 3 ", maxrec=10000)

Convert the results to a pandas data frame and show it.

In [ ]:
# data = results.to_table().to_pandas()
# data

Clean up, we don't need these anymore.

In [ ]:
# del results,data
# del service

### 2. Execute spatial search of the `src` catalog with the Butler.

Use [astropy.SkyCoord](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) to define the coordinates for the first variable star in the list above.

Since lists returned by the Butler are in random order, the coordinate used below might be different.

In [ ]:
targ_coord = SkyCoord(ra=62.1803311*u.deg, dec=-37.0137459*u.deg)

Use these coordinates to define the HTM ID spatial search region to pass to the Butler's `queryDatasets` function.

Find more details about sky pixelization with HTM and Butler spatial queries in Section 2.5 of notebook "04 Intro to Butler".

In [ ]:
pixelization = sphgeom.HtmPixelization(15)
htm_id = pixelization.index(sphgeom.UnitVector3d(sphgeom.LonLat.fromDegrees(targ_coord.ra.value, targ_coord.dec.value)))

# Obtain and print the scale to provide a sense of the size of the sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees()*3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

<br>

Query the `src` datasets for this HTM pixel via the Butler.
To keep things simple, just query for r-band data.

Find more details about the `src` catalog in the [DP0.1 DPDD](https://dp0-1.lsst.io/data-products-dp0-1/index.html#dp0-1-data-products-definition-document-dpdd).

In [ ]:
datasetRefs = registry.queryDatasets("src", htm20=htm_id, collections=collection, where="band in ('r')")

Extract the datasetRefs into a python list called `refs` to speed up access to the dataIds in Section 3.

In [ ]:
t1 = time.time()

ii = 0
refs = []
for i,ref in enumerate(datasetRefs):
    refs.append( ref )
    ii += 1
print(ii)

t2 = time.time()
print('t2-t1 = ',t2-t1)
del t1,t2,ii

### 3. Loop over search results and store `src` data in python lists.

For each query result (i.e., each element of `refs`), use the `dataId` to retrieve all sources for that query result.

Calculate the separation of each source from the target.
If the nearest source is within 1.5" of the target, it probably *is* the target.
Store the RA, Dec, r-band magnitude, and MJD for that source in a python list.

In [ ]:
### Instatiate empty lists
ra_arr = []
dec_arr = []
rmag_arr = []
mjd_arr = []

for i,d in enumerate(refs):
    
    if i < 10:

        t1 = time.time()

        # Use the butler to get all sources for this datasetRefs dataId
        did = d.dataId.full
        src = butler.get('src', dataId=did, collections=collection)
        t2 = time.time()
        print('i, len(src) = ',i,len(src))
        print('t2-t1 = ',t2-t1)

        # Get the separation of all sources from the target
        src_coords = SkyCoord(ra=src['coord_ra']*u.rad, dec=src['coord_dec']*u.rad)
        sep = src_coords.separation(targ_coord)
        t3 = time.time()
        print('t3-t2 = ', t3-t2)

        # If the nearest source is within 1.5", add it to our list
        if np.min(sep.arcsecond) < 1.5:
            sx = np.argmin(sep.arcsecond)
            ra_arr.append(src['coord_ra'][sx])
            dec_arr.append(src['coord_dec'][sx])

            # Calculate the AB magnitude from the calibrated flux in nJy
            # "calibrated flux" = base_PsfFlux_instFlux * base_localPhotoCalib
            rmag_arr.append( -2.5 * np.log10( src['base_PsfFlux_instFlux'][sx] * src['base_localPhotoCalib'][sx] ) + 31.4 )
            t4 = time.time()
            print('t4-t3 = ',t4-t3)

            # Retrieve the mjd date for this dataId from the image header for this dataid
            visit_info = butler.get('calexp.visitInfo', dataId=did, collections=collection)
            mjd_arr.append(visit_info.getDate().get(lsst.daf.base.DateTime.MJD))
            t5 = time.time()
            print('t5-t4 = ',t5-t4)

            del sx,visit_info

        del did,src,src_coords,sep

Plot the RA vs. Dec to show the dispersion in coordinate and get a sense of how well a 1.5 arcsecond limit for source association has worked in this case.
Seeing two clumps of points might indicate there is another nearby point-source.

In [ ]:
plt.plot(ra_arr, dec_arr, 'k.')
plt.plot(targ_coord.ra.rad,targ_coord.dec.rad, 'x', color='red')
plt.xlabel('ra')
plt.ylabel('dec')
plt.gca().invert_yaxis()
plt.minorticks_on()
plt.show()

In [ ]:
plt.plot(mjd_arr, rmag_arr, 'k.')
plt.xlabel('ra')
plt.ylabel('dec')
plt.gca().invert_yaxis()
plt.minorticks_on()
plt.show()